#### In this document, we show how to use the FISH (FAST InterSterllar HI) scripts to calibrate the FAST CRAFTS data.

### Import necessary packages

In [1]:
import os
import numpy as np
import sys
import gc
import multiprocessing as mp
sys.path.append('/bb8/lingrui/my_projects/FAST_pipeline/FISH_scripts/')

In [2]:
!pwd

/bb8/lingrui/my_projects/FAST_HINSA/SUN15_56_OTF


### Create soft links to the psr data and and spec data

In [3]:
os.system('rm org_psr org_spec')
os.system('ln -s /bb8/lingrui/FAST_data_all_softlinks/PT2021_0055_1/ org_psr')
os.system('ln -s /bb8/lingrui/FAST_data_all_softlinks/PT2021_0055_2/ org_spec')

0

### Prepare the output directories

In [4]:
Scan_name = ['Sun15_56_ra','Sun15_56_dec']

In [5]:
Beam_name = ['M01','M02','M03','M04','M05','M06','M07','M08','M09','M10','M11','M12','M13','M14','M15','M16','M17','M18','M19']

In [6]:
# ## run only once
# for i in Scan_name:
#     os.system('rm -rf {}'.format(i))      
#     os.system('mkdir {}'.format(i))
#     for j in Beam_name:
#         os.system('mkdir {}/{}'.format(i,j))
#         os.system('mkdir {}/{}/HighCAL'.format(i,j))
#         os.system('mkdir {}/{}/Calibrated'.format(i,j))

#### Generate PSR file list

In [7]:
rootwd = os.getcwd()

for i in Scan_name:
    for j in Beam_name:
        os.system('ls $PWD/org_psr/{0}/*/*{1}*.fits > {0}/{1}/mypsrlist_{0}_{1}'.format(i,j))

#### Extract cal (Noise_On and Noise_Off)

In [8]:
from extract_crafts_cal import extract_crafts_cal

rootwd = os.getcwd()

def target_extract_crafts_cal(i,j):
    os.chdir(rootwd+'/'+'{0}/{1}'.format(i,j))
    #print('Dealing with mypsrlist_{0}_{1}'.format(i,j))
    psrlist = np.loadtxt('mypsrlist_{0}_{1}'.format(i,j),dtype=str)
    for psrfits in psrlist:
        extract_crafts_cal(psrfits)

def run_process(i):
    process = []
    for j in Beam_name:
        process.append(mp.Process(target=target_extract_crafts_cal,args=(i,j)))
    [p.start() for p in process]
    [p.join()  for p in process]

for i in Scan_name:
    run_process(i)

Extracting CAL from /bb8/lingrui/my_projects/FAST_HINSA/SUN15_56_OTF/org_psr/Sun15_56_ra/20211008/Sun15_56_ra_OTF-M01_0001.fitsExtracting CAL from /bb8/lingrui/my_projects/FAST_HINSA/SUN15_56_OTF/org_psr/Sun15_56_ra/20211008/Sun15_56_ra_OTF-M02_0001.fits

Extracting CAL from /bb8/lingrui/my_projects/FAST_HINSA/SUN15_56_OTF/org_psr/Sun15_56_ra/20211008/Sun15_56_ra_OTF-M03_0001.fitsExtracting CAL from /bb8/lingrui/my_projects/FAST_HINSA/SUN15_56_OTF/org_psr/Sun15_56_ra/20211008/Sun15_56_ra_OTF-M04_0001.fits

Extracting CAL from /bb8/lingrui/my_projects/FAST_HINSA/SUN15_56_OTF/org_psr/Sun15_56_ra/20211008/Sun15_56_ra_OTF-M05_0001.fits
Extracting CAL from /bb8/lingrui/my_projects/FAST_HINSA/SUN15_56_OTF/org_psr/Sun15_56_ra/20211008/Sun15_56_ra_OTF-M06_0001.fits
Extracting CAL from /bb8/lingrui/my_projects/FAST_HINSA/SUN15_56_OTF/org_psr/Sun15_56_ra/20211008/Sun15_56_ra_OTF-M09_0001.fits
Extracting CAL from /bb8/lingrui/my_projects/FAST_HINSA/SUN15_56_OTF/org_psr/Sun15_56_ra/20211008/Sun15_

Extracting CAL from /bb8/lingrui/my_projects/FAST_HINSA/SUN15_56_OTF/org_psr/Sun15_56_ra/20211008/Sun15_56_ra_OTF-M11_0003.fits
Extracting CAL from /bb8/lingrui/my_projects/FAST_HINSA/SUN15_56_OTF/org_psr/Sun15_56_ra/20211008/Sun15_56_ra_OTF-M02_0003.fits
Extracting CAL from /bb8/lingrui/my_projects/FAST_HINSA/SUN15_56_OTF/org_psr/Sun15_56_ra/20211008/Sun15_56_ra_OTF-M13_0003.fits
Extracting CAL from /bb8/lingrui/my_projects/FAST_HINSA/SUN15_56_OTF/org_psr/Sun15_56_ra/20211008/Sun15_56_ra_OTF-M08_0003.fits
CAL SUCCESSFULLY EXTRACTED! HighCAL/Sun15_56_ra_OTF-M17_0002_211007_165212_cal.fits
CAL SUCCESSFULLY EXTRACTED! HighCAL/Sun15_56_ra_OTF-M10_0002_211007_165212_cal.fits
CAL SUCCESSFULLY EXTRACTED! HighCAL/Sun15_56_ra_OTF-M07_0002_211007_165212_cal.fits
CAL SUCCESSFULLY EXTRACTED! HighCAL/Sun15_56_ra_OTF-M18_0002_211007_165212_cal.fits
Extracting CAL from /bb8/lingrui/my_projects/FAST_HINSA/SUN15_56_OTF/org_psr/Sun15_56_ra/20211008/Sun15_56_ra_OTF-M17_0003.fits
CAL SUCCESSFULLY EXTRACT

CAL SUCCESSFULLY EXTRACTED! HighCAL/Sun15_56_ra_OTF-M08_0004_211007_165238_cal.fits
Extracting CAL from /bb8/lingrui/my_projects/FAST_HINSA/SUN15_56_OTF/org_psr/Sun15_56_ra/20211008/Sun15_56_ra_OTF-M08_0005.fits
CAL SUCCESSFULLY EXTRACTED! HighCAL/Sun15_56_ra_OTF-M16_0004_211007_165238_cal.fits
Extracting CAL from /bb8/lingrui/my_projects/FAST_HINSA/SUN15_56_OTF/org_psr/Sun15_56_ra/20211008/Sun15_56_ra_OTF-M16_0005.fits
CAL SUCCESSFULLY EXTRACTED! HighCAL/Sun15_56_ra_OTF-M10_0004_211007_165238_cal.fits
CAL SUCCESSFULLY EXTRACTED! HighCAL/Sun15_56_ra_OTF-M17_0004_211007_165238_cal.fits
Extracting CAL from /bb8/lingrui/my_projects/FAST_HINSA/SUN15_56_OTF/org_psr/Sun15_56_ra/20211008/Sun15_56_ra_OTF-M10_0005.fits
Extracting CAL from /bb8/lingrui/my_projects/FAST_HINSA/SUN15_56_OTF/org_psr/Sun15_56_ra/20211008/Sun15_56_ra_OTF-M17_0005.fits
CAL SUCCESSFULLY EXTRACTED! HighCAL/Sun15_56_ra_OTF-M07_0004_211007_165238_cal.fits
Extracting CAL from /bb8/lingrui/my_projects/FAST_HINSA/SUN15_56_OTF

Extracting CAL from /bb8/lingrui/my_projects/FAST_HINSA/SUN15_56_OTF/org_psr/Sun15_56_ra/20211008/Sun15_56_ra_OTF-M02_0007.fits
CAL SUCCESSFULLY EXTRACTED! HighCAL/Sun15_56_ra_OTF-M16_0006_211007_165304_cal.fits
Extracting CAL from /bb8/lingrui/my_projects/FAST_HINSA/SUN15_56_OTF/org_psr/Sun15_56_ra/20211008/Sun15_56_ra_OTF-M16_0007.fits
CAL SUCCESSFULLY EXTRACTED! HighCAL/Sun15_56_ra_OTF-M10_0006_211007_165304_cal.fits
Extracting CAL from /bb8/lingrui/my_projects/FAST_HINSA/SUN15_56_OTF/org_psr/Sun15_56_ra/20211008/Sun15_56_ra_OTF-M10_0007.fits
CAL SUCCESSFULLY EXTRACTED! HighCAL/Sun15_56_ra_OTF-M17_0006_211007_165304_cal.fits
CAL SUCCESSFULLY EXTRACTED! HighCAL/Sun15_56_ra_OTF-M07_0006_211007_165304_cal.fits
Extracting CAL from /bb8/lingrui/my_projects/FAST_HINSA/SUN15_56_OTF/org_psr/Sun15_56_ra/20211008/Sun15_56_ra_OTF-M17_0007.fits
Extracting CAL from /bb8/lingrui/my_projects/FAST_HINSA/SUN15_56_OTF/org_psr/Sun15_56_ra/20211008/Sun15_56_ra_OTF-M07_0007.fits
CAL SUCCESSFULLY EXTRACT

CAL SUCCESSFULLY EXTRACTED! HighCAL/Sun15_56_ra_OTF-M18_0007_211007_165317_cal.fits
Extracting CAL from /bb8/lingrui/my_projects/FAST_HINSA/SUN15_56_OTF/org_psr/Sun15_56_ra/20211008/Sun15_56_ra_OTF-M18_0008.fits
CAL SUCCESSFULLY EXTRACTED! HighCAL/Sun15_56_ra_OTF-M07_0008_211007_165330_cal.fits
Extracting CAL from /bb8/lingrui/my_projects/FAST_HINSA/SUN15_56_OTF/org_psr/Sun15_56_ra/20211008/Sun15_56_ra_OTF-M07_0009.fits
CAL SUCCESSFULLY EXTRACTED! HighCAL/Sun15_56_ra_OTF-M10_0008_211007_165330_cal.fits
CAL SUCCESSFULLY EXTRACTED! HighCAL/Sun15_56_ra_OTF-M16_0008_211007_165330_cal.fits
CAL SUCCESSFULLY EXTRACTED! HighCAL/Sun15_56_ra_OTF-M14_0008_211007_165330_cal.fits
CAL SUCCESSFULLY EXTRACTED! HighCAL/Sun15_56_ra_OTF-M06_0008_211007_165330_cal.fits
Extracting CAL from /bb8/lingrui/my_projects/FAST_HINSA/SUN15_56_OTF/org_psr/Sun15_56_ra/20211008/Sun15_56_ra_OTF-M10_0009.fits
Extracting CAL from /bb8/lingrui/my_projects/FAST_HINSA/SUN15_56_OTF/org_psr/Sun15_56_ra/20211008/Sun15_56_ra_OT

...

Extracting CAL from /bb8/lingrui/my_projects/FAST_HINSA/SUN15_56_OTF/org_psr/Sun15_56_dec/20211008/Sun15_56_dec_OTF-M04_0039.fits
Extracting CAL from /bb8/lingrui/my_projects/FAST_HINSA/SUN15_56_OTF/org_psr/Sun15_56_dec/20211008/Sun15_56_dec_OTF-M13_0042.fits
Extracting CAL from /bb8/lingrui/my_projects/FAST_HINSA/SUN15_56_OTF/org_psr/Sun15_56_dec/20211008/Sun15_56_dec_OTF-M03_0042.fits
CAL SUCCESSFULLY EXTRACTED! HighCAL/Sun15_56_dec_OTF-M08_0039_211007_171429_cal.fits
Extracting CAL from /bb8/lingrui/my_projects/FAST_HINSA/SUN15_56_OTF/org_psr/Sun15_56_dec/20211008/Sun15_56_dec_OTF-M14_0042.fits
Extracting CAL from /bb8/lingrui/my_projects/FAST_HINSA/SUN15_56_OTF/org_psr/Sun15_56_dec/20211008/Sun15_56_dec_OTF-M08_0040.fits
CAL SUCCESSFULLY EXTRACTED! HighCAL/Sun15_56_dec_OTF-M11_0038_211007_171416_cal.fits
Extracting CAL from /bb8/lingrui/my_projects/FAST_HINSA/SUN15_56_OTF/org_psr/Sun15_56_dec/20211008/Sun15_56_dec_OTF-M11_0039.fits
CAL SUCCESSFULLY EXTRACTED! HighCAL/Sun15_56_dec_O

Extracting CAL from /bb8/lingrui/my_projects/FAST_HINSA/SUN15_56_OTF/org_psr/Sun15_56_dec/20211008/Sun15_56_dec_OTF-M06_0044.fits
CAL SUCCESSFULLY EXTRACTED! HighCAL/Sun15_56_dec_OTF-M09_0042_211007_171508_cal.fits
CAL SUCCESSFULLY EXTRACTED! HighCAL/Sun15_56_dec_OTF-M19_0043_211007_171521_cal.fits
Extracting CAL from /bb8/lingrui/my_projects/FAST_HINSA/SUN15_56_OTF/org_psr/Sun15_56_dec/20211008/Sun15_56_dec_OTF-M16_0040.fits
Extracting CAL from /bb8/lingrui/my_projects/FAST_HINSA/SUN15_56_OTF/org_psr/Sun15_56_dec/20211008/Sun15_56_dec_OTF-M03_0044.fits
Extracting CAL from /bb8/lingrui/my_projects/FAST_HINSA/SUN15_56_OTF/org_psr/Sun15_56_dec/20211008/Sun15_56_dec_OTF-M09_0043.fits
Extracting CAL from /bb8/lingrui/my_projects/FAST_HINSA/SUN15_56_OTF/org_psr/Sun15_56_dec/20211008/Sun15_56_dec_OTF-M19_0044.fits
CAL SUCCESSFULLY EXTRACTED! HighCAL/Sun15_56_dec_OTF-M12_0045_211007_171546_cal.fits
Extracting CAL from /bb8/lingrui/my_projects/FAST_HINSA/SUN15_56_OTF/org_psr/Sun15_56_dec/20211

CAL SUCCESSFULLY EXTRACTED! HighCAL/Sun15_56_dec_OTF-M09_0044_211007_171534_cal.fits
CAL SUCCESSFULLY EXTRACTED! HighCAL/Sun15_56_dec_OTF-M01_0043_211007_171521_cal.fits
Extracting CAL from /bb8/lingrui/my_projects/FAST_HINSA/SUN15_56_OTF/org_psr/Sun15_56_dec/20211008/Sun15_56_dec_OTF-M01_0044.fits
Extracting CAL from /bb8/lingrui/my_projects/FAST_HINSA/SUN15_56_OTF/org_psr/Sun15_56_dec/20211008/Sun15_56_dec_OTF-M03_0046.fits
Extracting CAL from /bb8/lingrui/my_projects/FAST_HINSA/SUN15_56_OTF/org_psr/Sun15_56_dec/20211008/Sun15_56_dec_OTF-M09_0045.fits
CAL SUCCESSFULLY EXTRACTED! HighCAL/Sun15_56_dec_OTF-M16_0041_211007_171455_cal.fits
CAL SUCCESSFULLY EXTRACTED! HighCAL/Sun15_56_dec_OTF-M17_0043_211007_171521_cal.fits
Extracting CAL from /bb8/lingrui/my_projects/FAST_HINSA/SUN15_56_OTF/org_psr/Sun15_56_dec/20211008/Sun15_56_dec_OTF-M17_0044.fits
Extracting CAL from /bb8/lingrui/my_projects/FAST_HINSA/SUN15_56_OTF/org_psr/Sun15_56_dec/20211008/Sun15_56_dec_OTF-M16_0042.fits
CAL SUCCES

Extracting CAL from /bb8/lingrui/my_projects/FAST_HINSA/SUN15_56_OTF/org_psr/Sun15_56_dec/20211008/Sun15_56_dec_OTF-M17_0046.fits
Extracting CAL from /bb8/lingrui/my_projects/FAST_HINSA/SUN15_56_OTF/org_psr/Sun15_56_dec/20211008/Sun15_56_dec_OTF-M10_0047.fits
Extracting CAL from /bb8/lingrui/my_projects/FAST_HINSA/SUN15_56_OTF/org_psr/Sun15_56_dec/20211008/Sun15_56_dec_OTF-M15_0046.fits
Extracting CAL from /bb8/lingrui/my_projects/FAST_HINSA/SUN15_56_OTF/org_psr/Sun15_56_dec/20211008/Sun15_56_dec_OTF-M18_0049.fits
CAL SUCCESSFULLY EXTRACTED! HighCAL/Sun15_56_dec_OTF-M06_0049_211007_171638_cal.fits
Extracting CAL from /bb8/lingrui/my_projects/FAST_HINSA/SUN15_56_OTF/org_psr/Sun15_56_dec/20211008/Sun15_56_dec_OTF-M04_0045.fits
Extracting CAL from /bb8/lingrui/my_projects/FAST_HINSA/SUN15_56_OTF/org_psr/Sun15_56_dec/20211008/Sun15_56_dec_OTF-M02_0044.fits
Extracting CAL from /bb8/lingrui/my_projects/FAST_HINSA/SUN15_56_OTF/org_psr/Sun15_56_dec/20211008/Sun15_56_dec_OTF-M08_0046.fits
Extra

Extracting CAL from /bb8/lingrui/my_projects/FAST_HINSA/SUN15_56_OTF/org_psr/Sun15_56_dec/20211008/Sun15_56_dec_OTF-M01_0049.fits
CAL SUCCESSFULLY EXTRACTED! HighCAL/Sun15_56_dec_OTF-M17_0049_211007_171638_cal.fits
CAL SUCCESSFULLY EXTRACTED! HighCAL/Sun15_56_dec_OTF-M02_0046_211007_171559_cal.fits
CAL SUCCESSFULLY EXTRACTED! HighCAL/Sun15_56_dec_OTF-M15_0048_211007_171625_cal.fits
Extracting CAL from /bb8/lingrui/my_projects/FAST_HINSA/SUN15_56_OTF/org_psr/Sun15_56_dec/20211008/Sun15_56_dec_OTF-M02_0047.fits
Extracting CAL from /bb8/lingrui/my_projects/FAST_HINSA/SUN15_56_OTF/org_psr/Sun15_56_dec/20211008/Sun15_56_dec_OTF-M15_0049.fits
CAL SUCCESSFULLY EXTRACTED! HighCAL/Sun15_56_dec_OTF-M08_0048_211007_171625_cal.fits
CAL SUCCESSFULLY EXTRACTED! HighCAL/Sun15_56_dec_OTF-M15_0049_211007_171638_cal.fits
Extracting CAL from /bb8/lingrui/my_projects/FAST_HINSA/SUN15_56_OTF/org_psr/Sun15_56_dec/20211008/Sun15_56_dec_OTF-M08_0049.fits
CAL SUCCESSFULLY EXTRACTED! HighCAL/Sun15_56_dec_OTF-M0

#### Generate the cal_list

In [9]:
for i in Scan_name:
    for j in Beam_name:
        os.system('ls $PWD/{0}/{1}/HighCAL/* > {0}/{1}/mycallist_{0}_{1}'.format(i,j))

#### Generate Tsys_psr


In [10]:
from Tsys_cal import gaincal
!pwd

/bb8/lingrui/my_projects/FAST_HINSA/SUN15_56_OTF


In [11]:
Tcalfits = '/bb8/lingrui/my_projects/FAST_pipeline/FISH_scripts/Tcal/20201014/CAL.20201014.low.W.fits'
rootwd = os.getcwd()

def target_gaincal(i,j):
    os.chdir(rootwd+'/'+'{0}/{1}'.format(i,j))
    beam = int(j.split('M')[-1])
    print('Dealing with mycallist_{0}_{1}'.format(i,j))
    gaincal(callist='mycallist_{0}_{1}'.format(i,j),outname='Tsys_psr_{0}_{1}'.format(i,j),Tcalfits=Tcalfits,beam=beam,plot=True)
    print('Tsys_psr_{0}_{1} Created!'.format(i,j))
    gc.collect()

def run_process(j):
    process = []
    for i in Scan_name:
        process.append(mp.Process(target=target_gaincal,args=(i,j)))
    [p.start() for p in process]
    [p.join()  for p in process]

for j in Beam_name:
    run_process(j)

os.chdir(rootwd)

Dealing with mycallist_Sun15_56_dec_M01
Dealing with mycallist_Sun15_56_ra_M01
There are 2 polarizations in the Tcal noise.
There are 2 polarizations in the Tcal noise.
Tsys_psr_Sun15_56_ra_M01 Created!
Tsys_psr_Sun15_56_dec_M01 Created!
Dealing with mycallist_Sun15_56_dec_M02Dealing with mycallist_Sun15_56_ra_M02

There are 2 polarizations in the Tcal noise.
Tsys_psr_Sun15_56_dec_M02 Created!
There are 2 polarizations in the Tcal noise.
Tsys_psr_Sun15_56_ra_M02 Created!
Dealing with mycallist_Sun15_56_dec_M03Dealing with mycallist_Sun15_56_ra_M03

There are 2 polarizations in the Tcal noise.
There are 2 polarizations in the Tcal noise.
Tsys_psr_Sun15_56_dec_M03 Created!
Tsys_psr_Sun15_56_ra_M03 Created!
Dealing with mycallist_Sun15_56_dec_M04
Dealing with mycallist_Sun15_56_ra_M04
There are 2 polarizations in the Tcal noise.
There are 2 polarizations in the Tcal noise.
Tsys_psr_Sun15_56_ra_M04 Created!
Tsys_psr_Sun15_56_dec_M04 Created!
Dealing with mycallist_Sun15_56_dec_M05
Dealing 

/bb8/lingrui/my_projects/FAST_pipeline/FISH_scripts/Tsys_cal.py:86: RuntimeWarning: divide by zero encountered in true_divide
  Tpsr              = noise_data_interp/onoff_interp*cal_off ### Tsys_psr
/bb8/lingrui/my_projects/FAST_pipeline/FISH_scripts/Tsys_cal.py:86: RuntimeWarning: invalid value encountered in multiply
  Tpsr              = noise_data_interp/onoff_interp*cal_off ### Tsys_psr


There are 2 polarizations in the Tcal noise.


/bb8/lingrui/my_projects/FAST_pipeline/FISH_scripts/Tsys_cal.py:86: RuntimeWarning: divide by zero encountered in true_divide
  Tpsr              = noise_data_interp/onoff_interp*cal_off ### Tsys_psr


Tsys_psr_Sun15_56_ra_M07 Created!
Tsys_psr_Sun15_56_dec_M07 Created!
Dealing with mycallist_Sun15_56_ra_M08
Dealing with mycallist_Sun15_56_dec_M08
There are 2 polarizations in the Tcal noise.


...

Tsys_psr_Sun15_56_ra_M16 Created!
Tsys_psr_Sun15_56_dec_M16 Created!
Dealing with mycallist_Sun15_56_ra_M17
Dealing with mycallist_Sun15_56_dec_M17
There are 2 polarizations in the Tcal noise.


/bb8/lingrui/my_projects/FAST_pipeline/FISH_scripts/Tsys_cal.py:86: RuntimeWarning: divide by zero encountered in true_divide
  Tpsr              = noise_data_interp/onoff_interp*cal_off ### Tsys_psr
/bb8/lingrui/my_projects/FAST_pipeline/FISH_scripts/Tsys_cal.py:86: RuntimeWarning: invalid value encountered in multiply
  Tpsr              = noise_data_interp/onoff_interp*cal_off ### Tsys_psr


There are 2 polarizations in the Tcal noise.


/bb8/lingrui/my_projects/FAST_pipeline/FISH_scripts/Tsys_cal.py:86: RuntimeWarning: divide by zero encountered in true_divide
  Tpsr              = noise_data_interp/onoff_interp*cal_off ### Tsys_psr
/bb8/lingrui/my_projects/FAST_pipeline/FISH_scripts/Tsys_cal.py:86: RuntimeWarning: invalid value encountered in multiply
  Tpsr              = noise_data_interp/onoff_interp*cal_off ### Tsys_psr


Tsys_psr_Sun15_56_ra_M17 Created!
Tsys_psr_Sun15_56_dec_M17 Created!
Dealing with mycallist_Sun15_56_ra_M18
Dealing with mycallist_Sun15_56_dec_M18
There are 2 polarizations in the Tcal noise.
There are 2 polarizations in the Tcal noise.
Tsys_psr_Sun15_56_ra_M18 Created!
Tsys_psr_Sun15_56_dec_M18 Created!
Dealing with mycallist_Sun15_56_ra_M19
Dealing with mycallist_Sun15_56_dec_M19
There are 2 polarizations in the Tcal noise.
There are 2 polarizations in the Tcal noise.
Tsys_psr_Sun15_56_ra_M19 Created!
Tsys_psr_Sun15_56_dec_M19 Created!


#### Generate speclist

In [12]:
for i in Scan_name:
    for j in Beam_name:
        os.system('ls $PWD/org_spec/{0}/*/*{1}*N* > {0}/{1}/myspeclist_N_{0}_{1}'.format(i,j))

#### Apply cal

In [13]:
from apply_cal import applycal
import numpy
import glob

rootwd = os.getcwd()

def target_applycal(i,j):
    os.chdir(rootwd+'/'+'{}/{}'.format(i,j))
    print('Dealing with myspeclist_N_{}_{}'.format(i,j))
    tsysfits = glob.glob('Tsys_psr*.fits')[0]
    speclist = np.loadtxt('myspeclist_N_{}_{}'.format(i,j),dtype=str)
    for specfits in speclist:
        applycal(specfits,tsysfits,restfreq=1.420405751768,plot=True)
        print('Calibrated file of {} Created!'.format(specfits))
    gc.collect()

def run_process(i):
    process = []
    for j in Beam_name:
        process.append(mp.Process(target=target_applycal,args=(i,j)))
    [p.start() for p in process]
    [p.join()  for p in process]

for i in Scan_name:
    run_process(i)

Dealing with myspeclist_N_Sun15_56_ra_M01Dealing with myspeclist_N_Sun15_56_ra_M02

Dealing with myspeclist_N_Sun15_56_ra_M03
Matching the Tsys of /bb8/lingrui/my_projects/FAST_HINSA/SUN15_56_OTF/org_spec/Sun15_56_ra/20211008/Sun15_56_ra_OTF-M02_N_0001.fits and Tsys_psr_Sun15_56_ra_M02-M02_211007_165200to211007_170120.fits
Matching the Tsys of /bb8/lingrui/my_projects/FAST_HINSA/SUN15_56_OTF/org_spec/Sun15_56_ra/20211008/Sun15_56_ra_OTF-M01_N_0001.fits and Tsys_psr_Sun15_56_ra_M01-M01_211007_165200to211007_170120.fitsDealing with myspeclist_N_Sun15_56_ra_M04

Matching the Tsys of /bb8/lingrui/my_projects/FAST_HINSA/SUN15_56_OTF/org_spec/Sun15_56_ra/20211008/Sun15_56_ra_OTF-M03_N_0001.fits and Tsys_psr_Sun15_56_ra_M03-M03_211007_165200to211007_170120.fitsDealing with myspeclist_N_Sun15_56_ra_M05

Matching the Tsys of /bb8/lingrui/my_projects/FAST_HINSA/SUN15_56_OTF/org_spec/Sun15_56_ra/20211008/Sun15_56_ra_OTF-M04_N_0001.fits and Tsys_psr_Sun15_56_ra_M04-M04_211007_165200to211007_170120

/astrosoft/anaconda/anaconda3/lib/python3.8/site-packages/scipy/interpolate/interpolate.py:630: RuntimeWarning: invalid value encountered in subtract
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]
/astrosoft/anaconda/anaconda3/lib/python3.8/site-packages/scipy/interpolate/interpolate.py:633: RuntimeWarning: invalid value encountered in add
  y_new = slope*(x_new - x_lo)[:, None] + y_lo
/astrosoft/anaconda/anaconda3/lib/python3.8/site-packages/scipy/interpolate/interpolate.py:633: RuntimeWarning: invalid value encountered in add
  y_new = slope*(x_new - x_lo)[:, None] + y_lo
/astrosoft/anaconda/anaconda3/lib/python3.8/site-packages/scipy/interpolate/interpolate.py:630: RuntimeWarning: invalid value encountered in subtract
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]
/astrosoft/anaconda/anaconda3/lib/python3.8/site-packages/scipy/interpolate/interpolate.py:630: RuntimeWarning: invalid value encountered in subtract
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]
/astrosoft/anaconda

Calibrated file of /bb8/lingrui/my_projects/FAST_HINSA/SUN15_56_OTF/org_spec/Sun15_56_ra/20211008/Sun15_56_ra_OTF-M08_N_0001.fits Created!
Matching the Tsys of /bb8/lingrui/my_projects/FAST_HINSA/SUN15_56_OTF/org_spec/Sun15_56_ra/20211008/Sun15_56_ra_OTF-M08_N_0002.fits and Tsys_psr_Sun15_56_ra_M08-M08_211007_165200to211007_170120.fits
Calibrated file of /bb8/lingrui/my_projects/FAST_HINSA/SUN15_56_OTF/org_spec/Sun15_56_ra/20211008/Sun15_56_ra_OTF-M07_N_0001.fits Created!
Matching the Tsys of /bb8/lingrui/my_projects/FAST_HINSA/SUN15_56_OTF/org_spec/Sun15_56_ra/20211008/Sun15_56_ra_OTF-M07_N_0002.fits and Tsys_psr_Sun15_56_ra_M07-M07_211007_165200to211007_170120.fits
Calibrated file of /bb8/lingrui/my_projects/FAST_HINSA/SUN15_56_OTF/org_spec/Sun15_56_ra/20211008/Sun15_56_ra_OTF-M02_N_0001.fits Created!
Matching the Tsys of /bb8/lingrui/my_projects/FAST_HINSA/SUN15_56_OTF/org_spec/Sun15_56_ra/20211008/Sun15_56_ra_OTF-M02_N_0002.fits and Tsys_psr_Sun15_56_ra_M02-M02_211007_165200to21100

/astrosoft/anaconda/anaconda3/lib/python3.8/site-packages/scipy/interpolate/interpolate.py:633: RuntimeWarning: invalid value encountered in add
  y_new = slope*(x_new - x_lo)[:, None] + y_lo
/home/lingrui/.local/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1218: RuntimeWarning: All-NaN slice encountered
  r, k = function_base._ureduce(a, func=_nanmedian, axis=axis, out=out,
/home/lingrui/.local/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1218: RuntimeWarning: All-NaN slice encountered
  r, k = function_base._ureduce(a, func=_nanmedian, axis=axis, out=out,
/home/lingrui/.local/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1218: RuntimeWarning: All-NaN slice encountered
  r, k = function_base._ureduce(a, func=_nanmedian, axis=axis, out=out,
/home/lingrui/.local/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1218: RuntimeWarning: All-NaN slice encountered
  r, k = function_base._ureduce(a, func=_nanmedian, axis=axis, out=out,
/home/lingrui/.local/lib/pyt

Calibrated file of /bb8/lingrui/my_projects/FAST_HINSA/SUN15_56_OTF/org_spec/Sun15_56_ra/20211008/Sun15_56_ra_OTF-M13_N_0001.fits Created!
Matching the Tsys of /bb8/lingrui/my_projects/FAST_HINSA/SUN15_56_OTF/org_spec/Sun15_56_ra/20211008/Sun15_56_ra_OTF-M13_N_0002.fits and Tsys_psr_Sun15_56_ra_M13-M13_211007_165200to211007_170120.fits
Calibrated file of /bb8/lingrui/my_projects/FAST_HINSA/SUN15_56_OTF/org_spec/Sun15_56_ra/20211008/Sun15_56_ra_OTF-M08_N_0002.fits Created!
Calibrated file of /bb8/lingrui/my_projects/FAST_HINSA/SUN15_56_OTF/org_spec/Sun15_56_ra/20211008/Sun15_56_ra_OTF-M07_N_0002.fits Created!
Calibrated file of /bb8/lingrui/my_projects/FAST_HINSA/SUN15_56_OTF/org_spec/Sun15_56_ra/20211008/Sun15_56_ra_OTF-M03_N_0001.fits Created!
Matching the Tsys of /bb8/lingrui/my_projects/FAST_HINSA/SUN15_56_OTF/org_spec/Sun15_56_ra/20211008/Sun15_56_ra_OTF-M03_N_0002.fits and Tsys_psr_Sun15_56_ra_M03-M03_211007_165200to211007_170120.fits
Calibrated file of /bb8/lingrui/my_projects/FA

...

Calibrated file of /bb8/lingrui/my_projects/FAST_HINSA/SUN15_56_OTF/org_spec/Sun15_56_dec/20211008/Sun15_56_dec_OTF-M12_N_0002.fits Created!
Calibrated file of /bb8/lingrui/my_projects/FAST_HINSA/SUN15_56_OTF/org_spec/Sun15_56_dec/20211008/Sun15_56_dec_OTF-M09_N_0002.fits Created!


/home/lingrui/.local/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1218: RuntimeWarning: All-NaN slice encountered
  r, k = function_base._ureduce(a, func=_nanmedian, axis=axis, out=out,
/home/lingrui/.local/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1218: RuntimeWarning: All-NaN slice encountered
  r, k = function_base._ureduce(a, func=_nanmedian, axis=axis, out=out,
/home/lingrui/.local/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1218: RuntimeWarning: All-NaN slice encountered
  r, k = function_base._ureduce(a, func=_nanmedian, axis=axis, out=out,
/home/lingrui/.local/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1218: RuntimeWarning: All-NaN slice encountered
  r, k = function_base._ureduce(a, func=_nanmedian, axis=axis, out=out,


Calibrated file of /bb8/lingrui/my_projects/FAST_HINSA/SUN15_56_OTF/org_spec/Sun15_56_dec/20211008/Sun15_56_dec_OTF-M08_N_0002.fits Created!


/home/lingrui/.local/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1218: RuntimeWarning: All-NaN slice encountered
  r, k = function_base._ureduce(a, func=_nanmedian, axis=axis, out=out,
/home/lingrui/.local/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1218: RuntimeWarning: All-NaN slice encountered
  r, k = function_base._ureduce(a, func=_nanmedian, axis=axis, out=out,
/home/lingrui/.local/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1218: RuntimeWarning: All-NaN slice encountered
  r, k = function_base._ureduce(a, func=_nanmedian, axis=axis, out=out,
/home/lingrui/.local/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1218: RuntimeWarning: All-NaN slice encountered
  r, k = function_base._ureduce(a, func=_nanmedian, axis=axis, out=out,


Calibrated file of /bb8/lingrui/my_projects/FAST_HINSA/SUN15_56_OTF/org_spec/Sun15_56_dec/20211008/Sun15_56_dec_OTF-M13_N_0002.fits Created!
Calibrated file of /bb8/lingrui/my_projects/FAST_HINSA/SUN15_56_OTF/org_spec/Sun15_56_dec/20211008/Sun15_56_dec_OTF-M02_N_0002.fits Created!
Calibrated file of /bb8/lingrui/my_projects/FAST_HINSA/SUN15_56_OTF/org_spec/Sun15_56_dec/20211008/Sun15_56_dec_OTF-M19_N_0002.fits Created!
Calibrated file of /bb8/lingrui/my_projects/FAST_HINSA/SUN15_56_OTF/org_spec/Sun15_56_dec/20211008/Sun15_56_dec_OTF-M04_N_0002.fits Created!
Calibrated file of /bb8/lingrui/my_projects/FAST_HINSA/SUN15_56_OTF/org_spec/Sun15_56_dec/20211008/Sun15_56_dec_OTF-M03_N_0002.fits Created!
Calibrated file of /bb8/lingrui/my_projects/FAST_HINSA/SUN15_56_OTF/org_spec/Sun15_56_dec/20211008/Sun15_56_dec_OTF-M15_N_0002.fits Created!
Calibrated file of /bb8/lingrui/my_projects/FAST_HINSA/SUN15_56_OTF/org_spec/Sun15_56_dec/20211008/Sun15_56_dec_OTF-M05_N_0002.fits Created!
Calibrated fi

#### xyz2radec

In [14]:
for i in Scan_name:
    for j in Beam_name:
        os.system('ls $PWD/{0}/{1}/Calibrated/*calibrated.fits > {0}/{1}/myspeclist_N_{0}_{1}_calibrated'.format(i,j))

from xyz2celestial import xyz2celestial
import glob
import pandas as pd

os.system('ln -s /bb8/lingrui/FAST_data_all_softlinks/PT2021_0055KY PT2021_0055_KYKY')

0

In [17]:
rootwd = os.getcwd()

def target_xyz2celestial(i,j,xyztable):
    os.chdir(rootwd+'/'+'{}/{}'.format(i,j))
    print('Dealing with myspeclist_N_{0}_{1}_calibrated'.format(i,j))
    beam = int(j.split('M')[-1])
    speclist = np.loadtxt('myspeclist_N_{0}_{1}_calibrated'.format(i,j),dtype=str)
    for specfits in speclist:
        xyz2celestial(specfits,xyztable,beam=beam,plot=False)

def run_process(i,xyztable):
    process = []
    for j in Beam_name:
        process.append(mp.Process(target=target_xyz2celestial,args=(i,j,xyztable)))
    [p.start() for p in process]
    [p.join()  for p in process]

for i in Scan_name:
    table = glob.glob(rootwd+'/*KY/'+'{}*.xlsx'.format(i))[0]
    xyztable = pd.read_excel(table,sheet_name=0)
    run_process(i,xyztable)

Dealing with myspeclist_N_Sun15_56_ra_M01_calibrated
Dealing with myspeclist_N_Sun15_56_ra_M02_calibrated
Dealing with myspeclist_N_Sun15_56_ra_M03_calibrated
Dealing with myspeclist_N_Sun15_56_ra_M04_calibrated
Dealing with myspeclist_N_Sun15_56_ra_M05_calibrated

Dealing with myspeclist_N_Sun15_56_ra_M06_calibratedDealing with myspeclist_N_Sun15_56_ra_M09_calibrated
Dealing with myspeclist_N_Sun15_56_ra_M10_calibrated
Dealing with myspeclist_N_Sun15_56_ra_M11_calibratedDealing with myspeclist_N_Sun15_56_ra_M07_calibratedDealing with myspeclist_N_Sun15_56_ra_M08_calibrated


Dealing with myspeclist_N_Sun15_56_ra_M12_calibrated
Dealing with myspeclist_N_Sun15_56_ra_M13_calibrated
Dealing with myspeclist_N_Sun15_56_ra_M14_calibrated
Dealing with myspeclist_N_Sun15_56_ra_M15_calibrated
Dealing with myspeclist_N_Sun15_56_ra_M16_calibrated
Dealing with myspeclist_N_Sun15_56_ra_M17_calibrated
Dealing with myspeclist_N_Sun15_56_ra_M18_calibrated
Dealing with myspeclist_N_Sun15_56_ra_M19_cali

/home/lingrui/.local/lib/python3.8/site-packages/astropy/time/utils.py:179: RuntimeWarning: overflow encountered in double_scalars
  c = 134217729. * a  # 2**27+1.
/home/lingrui/.local/lib/python3.8/site-packages/astropy/time/utils.py:181: RuntimeWarning: invalid value encountered in double_scalars
  ah = c - abig
/home/lingrui/.local/lib/python3.8/site-packages/astropy/time/utils.py:179: RuntimeWarning: overflow encountered in double_scalars
  c = 134217729. * a  # 2**27+1.
/home/lingrui/.local/lib/python3.8/site-packages/astropy/time/utils.py:181: RuntimeWarning: invalid value encountered in double_scalars
  ah = c - abig
/home/lingrui/.local/lib/python3.8/site-packages/astropy/time/utils.py:179: RuntimeWarning: overflow encountered in double_scalars
  c = 134217729. * a  # 2**27+1.
/home/lingrui/.local/lib/python3.8/site-packages/astropy/time/utils.py:181: RuntimeWarning: invalid value encountered in double_scalars
  ah = c - abig
/home/lingrui/.local/lib/python3.8/site-packages/ast

Dealing with myspeclist_N_Sun15_56_dec_M01_calibrated
Dealing with myspeclist_N_Sun15_56_dec_M02_calibrated
Dealing with myspeclist_N_Sun15_56_dec_M03_calibrated
Dealing with myspeclist_N_Sun15_56_dec_M04_calibrated
Dealing with myspeclist_N_Sun15_56_dec_M05_calibrated
Dealing with myspeclist_N_Sun15_56_dec_M06_calibrated
Dealing with myspeclist_N_Sun15_56_dec_M09_calibrated
Dealing with myspeclist_N_Sun15_56_dec_M10_calibratedDealing with myspeclist_N_Sun15_56_dec_M07_calibrated

Dealing with myspeclist_N_Sun15_56_dec_M11_calibratedDealing with myspeclist_N_Sun15_56_dec_M08_calibrated

Dealing with myspeclist_N_Sun15_56_dec_M12_calibrated
Dealing with myspeclist_N_Sun15_56_dec_M13_calibrated
Dealing with myspeclist_N_Sun15_56_dec_M14_calibrated
Dealing with myspeclist_N_Sun15_56_dec_M15_calibrated
Dealing with myspeclist_N_Sun15_56_dec_M16_calibrated
Dealing with myspeclist_N_Sun15_56_dec_M17_calibrated
Dealing with myspeclist_N_Sun15_56_dec_M18_calibrated
Dealing with myspeclist_N_Su

/home/lingrui/.local/lib/python3.8/site-packages/astropy/time/utils.py:179: RuntimeWarning: overflow encountered in double_scalars
  c = 134217729. * a  # 2**27+1.
/home/lingrui/.local/lib/python3.8/site-packages/astropy/time/utils.py:181: RuntimeWarning: invalid value encountered in double_scalars
  ah = c - abig
/home/lingrui/.local/lib/python3.8/site-packages/astropy/time/utils.py:179: RuntimeWarning: overflow encountered in double_scalars
  c = 134217729. * a  # 2**27+1.
/home/lingrui/.local/lib/python3.8/site-packages/astropy/time/utils.py:181: RuntimeWarning: invalid value encountered in double_scalars
  ah = c - abig
/home/lingrui/.local/lib/python3.8/site-packages/astropy/time/utils.py:179: RuntimeWarning: overflow encountered in double_scalars
  c = 134217729. * a  # 2**27+1.
/home/lingrui/.local/lib/python3.8/site-packages/astropy/time/utils.py:181: RuntimeWarning: invalid value encountered in double_scalars
  ah = c - abig
/home/lingrui/.local/lib/python3.8/site-packages/ast

#### Ta to Tmb

In [ ]:
# for i in Scan_name:
#     for j in Beam_name:
#         os.system('ls $PWD/{0}/{1}/Calibrated/*calibrated_icrs.fits > {0}/{1}/myspeclist_N_{0}_{1}_calibrated_icrs'.format(i,j))

In [ ]:
# from Ta2Tmb import Ta2Tmb
# import glob
# import pandas as pd

In [ ]:
# help(Ta2Tmb)

In [ ]:
# rootwd = os.getcwd()

# def target_Ta2Tmb(i,j):
#     os.chdir(rootwd+'/'+'{}/{}'.format(i,j))
#     print('Dealing with myspeclist_N_{0}_{1}_calibrated_icrs'.format(i,j))
#     beam = int(j.split('M')[-1])
#     speclist = np.loadtxt('myspeclist_N_{0}_{1}_calibrated_icrs'.format(i,j),dtype=str)
#     for specfits in speclist:
#         Ta2Tmb(specfits,beam=beam,restfreq=1420.405751768)

# def run_process(i):
#     process = []
#     for j in Beam_name:
#         process.append(mp.Process(target=target_Ta2Tmb,args=(i,j)))
#     [p.start() for p in process]
#     [p.join()  for p in process]

# for i in Scan_name:
#     run_process(i)

#### Gridding 

In [18]:
# os.system('ls $PWD/Sun17_A_00/*/Calibrated/*_calibrated_icrs.fits \
#               $PWD/Sun17_A_01/*/Calibrated/*_calibrated_icrs.fits \
#               $PWD/Sun17_A_02/*/Calibrated/*_calibrated_icrs.fits \
#               $PWD/Sun17_A_03/*/Calibrated/*_calibrated_icrs.fits \
#               $PWD/Sun17_A_04/*/Calibrated/*_calibrated_icrs.fits > mygridlist')

os.system('ls $PWD/*/*/Calibrated/*_calibrated_icrs.fits > mygridlist_all')

0

In [19]:
from gridding_with_doppler_tracking import gridding_with_doppler_tracking

speclist = 'mygridlist_all'
center = [42.156000,58.474000]
outname = 'Sun15_56_otf_doppler.fits'
gridding_with_doppler_tracking(speclist=speclist, center=center, imsize=[10, 10], cell=1/60, rest_freq=1.420405751768, vmin=-300, vmax=300, dv=0.15, p=[0,1], outname=outname, overwrite=True)

{'NAXIS': 2, 'NAXIS1': 10, 'NAXIS2': 10, 'CTYPE1': 'RA---SIN', 'CTYPE2': 'DEC--SIN', 'CUNIT1': 'deg', 'CUNIT2': 'deg', 'CDELT1': -0.016666666666666666, 'CDELT2': 0.016666666666666666, 'CRPIX1': 5.0, 'CRPIX2': 5.0, 'CRVAL1': 42.156, 'CRVAL2': 58.474}
Gridding /bb8/lingrui/my_projects/FAST_HINSA/SUN15_56_OTF/Sun15_56_dec/M01/Calibrated/Sun15_56_dec_OTF-M01_N_0001_211007_170620to211007_171312_calibrated_icrs.fits
Doing Doppler Tracking!


/home/lingrui/.local/lib/python3.8/site-packages/astropy/time/utils.py:179: RuntimeWarning: overflow encountered in double_scalars
  c = 134217729. * a  # 2**27+1.
/home/lingrui/.local/lib/python3.8/site-packages/astropy/time/utils.py:181: RuntimeWarning: invalid value encountered in double_scalars
  ah = c - abig


Gridding /bb8/lingrui/my_projects/FAST_HINSA/SUN15_56_OTF/Sun15_56_dec/M01/Calibrated/Sun15_56_dec_OTF-M01_N_0002_211007_171312to211007_171640_calibrated_icrs.fits
Doing Doppler Tracking!


Gridding /bb8/lingrui/my_projects/FAST_HINSA/SUN15_56_OTF/Sun15_56_dec/M02/Calibrated/Sun15_56_dec_OTF-M02_N_0001_211007_170620to211007_171312_calibrated_icrs.fits
Doing Doppler Tracking!


Gridding /bb8/lingrui/my_projects/FAST_HINSA/SUN15_56_OTF/Sun15_56_dec/M02/Calibrated/Sun15_56_dec_OTF-M02_N_0002_211007_171312to211007_171640_calibrated_icrs.fits
Doing Doppler Tracking!


...

Gridding /bb8/lingrui/my_projects/FAST_HINSA/SUN15_56_OTF/Sun15_56_ra/M17/Calibrated/Sun15_56_ra_OTF-M17_N_0002_211007_165852to211007_170120_calibrated_icrs.fits
Doing Doppler Tracking!


Gridding /bb8/lingrui/my_projects/FAST_HINSA/SUN15_56_OTF/Sun15_56_ra/M18/Calibrated/Sun15_56_ra_OTF-M18_N_0001_211007_165200to211007_165852_calibrated_icrs.fits
Doing Doppler Tracking!


Gridding /bb8/lingrui/my_projects/FAST_HINSA/SUN15_56_OTF/Sun15_56_ra/M18/Calibrated/Sun15_56_ra_OTF-M18_N_0002_211007_165852to211007_170120_calibrated_icrs.fits
Doing Doppler Tracking!


Gridding /bb8/lingrui/my_projects/FAST_HINSA/SUN15_56_OTF/Sun15_56_ra/M19/Calibrated/Sun15_56_ra_OTF-M19_N_0001_211007_165200to211007_165852_calibrated_icrs.fits
Doing Doppler Tracking!


Gridding /bb8/lingrui/my_projects/FAST_HINSA/SUN15_56_OTF/Sun15_56_ra/M19/Calibrated/Sun15_56_ra_OTF-M19_N_0002_211007_165852to211007_170120_calibrated_icrs.fits
Doing Doppler Tracking!


# Wideband continuum

#### xyz2radec

In [ ]:
# from xyz2celestial import xyz2celestial
# import glob
# import pandas as pd

# os.system('ln -s /TheForce/fast_data/PT2021_0055_KY PT2021_0055_KY')

In [ ]:
# rootwd = os.getcwd()

# def target_xyz2celestial(i,j,xyztable):
#     os.chdir(rootwd+'/'+'{}/{}'.format(i,j))
#     beam = int(j.split('M')[-1])
#     specfits = glob.glob('Tsys_psr*.fits')[0]
#     print('Dealing with '+specfits)
#     xyz2celestial(specfits,xyztable,beam=beam,plot=False,psrflag=True)

# def run_process(i,xyztable):
#     process = []
#     for j in Beam_name:
#         process.append(mp.Process(target=target_xyz2celestial,args=(i,j,xyztable)))
#     [p.start() for p in process]
#     [p.join()  for p in process]

# for i in Scan_name:
#     table = glob.glob(rootwd+'/*KY/'+'{}*.xlsx'.format(i))[0]
#     xyztable = pd.read_excel(table,sheet_name=0)
#     run_process(i,xyztable)

#### gridding

In [ ]:
# os.system('ls $PWD/*/*/Tsys_psr*icrs.fits > mygridcont_list')

In [ ]:
# from gridding_with_doppler_tracking import gridding_with_doppler_tracking

# speclist = 'mygridcont_list'
# center = [284.5,6.5]
# outname = 'Sun17_A_scans_all_doppler_cont.fits'
# vmin = (1.4204-1.45)/1.4204*3e5
# vmax = (1.4204-1.05)/1.4204*3e5
# dv   = 500e-3/1024/1.4204*3e5 
# gridding_with_doppler_tracking(speclist=speclist, center=center, imsize=[360, 360], \
#                                cell=1/60, rest_freq=1.420405751768, vmin=vmin, vmax=vmax, \
#                                dv=dv, p=0,  outname=outname, psrflag=True, overwrite=False)